In [2]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix

np.set_printoptions(threshold=np.inf)
np.set_printoptions(suppress=True)

# Load your data
measurement = np.load('../dataset/meas_symm_1.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs']

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

# Extract data for ALICE and EVE channels
alice_CIRs = data_cir[:, alice_channel, :, :]  # Use all available samples
eve_CIRs = data_cir[:, eve_channel, :, :]

# ALICE features - real, imaginary, magnitude
alice_real = alice_CIRs[:, :, 0]
alice_imag = alice_CIRs[:, :, 1]
alice_magnitude = np.abs(alice_real + 1j * alice_imag)
alice_features = np.hstack((alice_real, alice_imag, alice_magnitude))

# EVE features - real, imaginary, magnitude
eve_real = eve_CIRs[:, :, 0]
eve_imag = eve_CIRs[:, :, 1]
eve_magnitude = np.abs(eve_real + 1j * eve_imag)
eve_features = np.hstack((eve_real, eve_imag, eve_magnitude))

# Reshape data to flatten each sample
# alice_atoms = alice_features.reshape(alice_features.shape[0], -1)
# eve_atoms = eve_features.reshape(eve_features.shape[0], -1)

# Create labels
alice_labels = np.zeros(alice_features.shape[0])  # Label '0' for Alice
eve_labels = np.ones(eve_features.shape[0])       # Label '1' for Eve

# Combine data and labels
atoms = np.vstack((alice_features, eve_features))
true_labels = np.hstack((alice_labels, eve_labels))

# Define cross-validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

accuracies = []
confusion_matrices = []

def find_sparse_coefficients(tSample, D, n_nonzero_coefs=10):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

def calculate_residual(tSample, D, coefficients, class_indices):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]
    reconstructed_signal = D @ coef_class
    residual = np.linalg.norm(tSample - reconstructed_signal)
    return residual

def classify_signal(tSample, D, trainLabel):
    coefficients = find_sparse_coefficients(tSample, D)
    residuals = []
    unique_classes = np.unique(trainLabel)
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]
        residual = calculate_residual(tSample, D, coefficients, class_indices)
        residuals.append(residual)
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class

# Cross-validation loop
for fold, (train_index, test_index) in enumerate(skf.split(atoms, true_labels)):
    # print(f'{fold}')
    # print(f"Train Index: {train_index}")
    # print(f"Test Index: {test_index}")
    # print(f"\nFold {fold + 1}")
    # Split the data
    trainData, testData = atoms[train_index], atoms[test_index]
    trainLabel, testLabel = true_labels[train_index], true_labels[test_index]
    
    # Form the Dictionary D
    D = trainData.T

    # Classify Test Data
    predictions = []
    for testSample in testData:
        predicted_class = classify_signal(testSample, D, trainLabel)
        predictions.append(predicted_class)
    predictions = np.array(predictions)
    
    accuracy = accuracy_score(testLabel, predictions)
    accuracies.append(accuracy)
    print(f"Accuracy: {accuracy * 100:.2f}%")
    
    
    cm = confusion_matrix(testLabel, predictions, labels=[0, 1])
    confusion_matrices.append(cm)


average_accuracy = np.mean(accuracies)
print(f"\nAverage Classification Accuracy over {n_splits} folds: {average_accuracy * 100:.2f}%")


sum_cm = np.sum(confusion_matrices, axis=0)
tn, fp, fn, tp = sum_cm.ravel()

print(f"\nTotal Confusion Matrix over {n_splits} folds:")
print(f"True Positives (tp): {tp}")
print(f"True Negatives (tn): {tn}")
print(f"False Positives (fp): {fp}")
print(f"False Negatives (fn): {fn}")

# Recalculate MDR, FAR, AR using summed confusion matrix
MDR = fp / (fp + tn) if (fp + tn) > 0 else 0
FAR = fn / (fn + tp) if (fn + tp) > 0 else 0
gamma = (tp + fn) / (tn + fp) if (tn + fp) > 0 else 0
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp)) if ((tp + fn) + gamma * (tn + fp)) > 0 else 0

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")


Accuracy: 66.29%
Accuracy: 65.73%
Accuracy: 64.61%
Accuracy: 65.22%
Accuracy: 64.83%

Average Classification Accuracy over 5 folds: 65.34%

Total Confusion Matrix over 5 folds:
True Positives (tp): 8179
True Negatives (tn): 4623
False Positives (fp): 5174
False Negatives (fn): 1618
MDR: 0.5281208533224456
FAR: 0.1651525977340002
AR: 0.6533632744717771
